In [1]:
import sys
import os
from groq import Groq
sys.path.append(os.path.dirname(os.path.abspath("./")))
from src.tools import model_tools
from src.tools.base import Tool
from configs.config import GROQ_API_KEY
from dotenv import load_dotenv
from warnings import filterwarnings

filterwarnings("ignore")

Loaded env variables: False


In [2]:
load_dotenv("../secrets/environments/local.env")

True

In [3]:
# get tools
tool_list = []
for tool in model_tools.__all__:
    tool_list.append(Tool.from_function(function=getattr(model_tools, tool)))

openai_tools = [tool.to_openai_tool() for tool in tool_list]

In [4]:
openai_tools

[{'type': 'function',
  'function': {'name': 'multiply',
   'description': 'multiply(a: int, b: int) -> int\nMultiplies two integers and returns the result integer.\n\n    Args:\n        a (int): The first integer.\n        b (int): The second integer.\n\n    Returns:\n        int: The product of a and b.\n    ',
   'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
     'b': {'title': 'B', 'type': 'integer'}},
    'required': ['a', 'b'],
    'type': 'object'}}},
 {'type': 'function',
  'function': {'name': 'add',
   'description': 'add(a: int, b: int) -> int\nAdd two integers and returns the result integer.\n\n    Args:\n        a (int): The first integer.\n        b (int): The second integer.\n\n    Returns:\n        int: The sum of a and b\n    ',
   'parameters': {'properties': {'a': {'title': 'A', 'type': 'integer'},
     'b': {'title': 'B', 'type': 'integer'}},
    'required': ['a', 'b'],
    'type': 'object'}}},
 {'type': 'function',
  'function': {'name': 'su

In [5]:
model_service = Groq(
    timeout=120.0, default_headers={"Content-Type": "application/json"}
)

In [6]:
json_mode = False
response = model_service.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "What is 1 + 1?"},
        {
            "role": "assistant",
            "content": "",
            "tool_calls": [
                {
                    "id": "call_dhqx",
                    "type": "function",
                    "function": {"name": "add", "arguments": '{"a": 1, "b": 1}'},
                }
            ],
        },
        {"tool_call_id": "call_dhqx", "role": "tool", "name": "add", "content": "2"},
    ],
    tools=openai_tools,
    tool_choice="auto",
    response_format={"type": "json_object"} if json_mode else None,
    stop=None,
)

In [7]:
response.to_dict()

{'id': 'chatcmpl-6d9276ee-042b-4d7e-96d7-77145aa6a84d',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': 'The answer is 2.', 'role': 'assistant'}}],
 'created': 1726736276,
 'model': 'llama-3.1-70b-versatile',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_b6828be2c9',
 'usage': {'completion_tokens': 7,
  'prompt_tokens': 916,
  'total_tokens': 923,
  'completion_time': 0.028147274,
  'prompt_time': 0.265775336,
  'queue_time': 0.013023072000000024,
  'total_time': 0.29392261},
 'x_groq': {'id': 'req_01j84r16qgedj8s3bjqqtgjdt3'}}

In [5]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath("./")))
from src.models.groq_model import GroqModel

model = GroqModel(model="llama-3.1-70b-versatile", is_tool_use_model=True, system_prompt="You are a helpful chat assistant")

In [8]:
model.chat(
    {"role": "user", "content": "What is 2 + 3 + 4?"},
    tools=tool_list,
)


[MODEL]: ChatCompletion(id='chatcmpl-368ea540-ab28-4152-b722-5d94df44de50', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_rfp7', function=Function(arguments='{"a": "2", "b": "3"}', name='add'), type='function'), ChatCompletionMessageToolCall(id='call_a75s', function=Function(arguments='{"a": "5", "b": "4"}', name='add'), type='function')]))], created=1726736754, model='llama-3.1-70b-versatile', object='chat.completion', system_fingerprint='fp_b3ae7e594e', usage=CompletionUsage(completion_tokens=37, prompt_tokens=891, total_tokens=928, completion_time=0.148823295, prompt_time=0.255935641, queue_time=0.005115148000000014, total_time=0.404758936), x_groq={'id': 'req_01j84rft92fqwsqag8dd9egx1c'})



{'role': 'assistant',
 'tool_calls': [{'id': 'call_rfp7',
   'function': {'arguments': '{"a": "2", "b": "3"}', 'name': 'add'},
   'type': 'function'},
  {'id': 'call_a75s',
   'function': {'arguments': '{"a": "5", "b": "4"}', 'name': 'add'},
   'type': 'function'}]}